### Импорты

In [1]:
import os

import numpy as np
import pandas as pd
import re
import time
from tqdm.notebook import tqdm
from datetime import date, datetime

import json

import warnings
warnings.filterwarnings('ignore')

In [4]:
path = '../data/channels/channels.csv'
channels_info = pd.read_csv(path)
channels_info.head()

,channel_id,name,subscribers,link,description,create_date,message_cnt
0,0,Тинькофф Инвестиции,670825,https://t.me/tinkoff_invest_official,Официальный канал Тинькофф.,2019-09-04 15:50:32,44.0
1,1,Альфа-Инвестиции,415942,https://t.me/alfa_investments,Официальный канал Альфа-Инвестиций с инвестиде...,2021-06-18 14:04:01,384.0
2,2,Мои Инвестиции,286117,https://t.me/omyinvestments,"Инвестидеи, обзоры рынка и многое другое от лу...",2019-09-12 12:53:25,11.0
3,3,bitkogan,284405,https://t.me/bitkogan,Авторский канал Евгения Когана,2018-01-14 14:46:07,1777.0
4,4,СберИнвестиции,232751,https://t.me/SberInvestments,"Канал СберИнвестиции, в котором мы рассказывае...",2020-07-24 16:15:43,419.0


In [5]:
path = '../data/messages/messages.csv'
messages_df = pd.read_csv(path)
messages_df.head()

,message_id,channel_id,tg_message_id,date,text,views,forwards,reactions
0,552,1,4332,2023-11-02 12:31:28,"Центробанку, скорее всего, придётся поднять ст...",110394,401,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
1,551,1,4333,2023-11-02 14:33:01,Сбербанк продолжает рост: результаты по междун...,110495,76,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
2,550,1,4337,2023-11-02 15:56:40,Плохие новости сегодня вечером. СПБ Биржа попа...,125490,469,"[{'_': 'ReactionCount', 'reaction': '😢', 'coun..."
3,549,1,4338,2023-11-03 07:59:27,📉 Вчера на российском рынке акций прошли распр...,323274,41,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
4,548,1,4339,2023-11-03 10:44:50,У Норникеля сразу две приятные новости: компан...,315471,240,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."


In [6]:
messages_df['reactions'] = messages_df['reactions'].apply(lambda x: eval(x) if type(x) == str else None)

In [7]:
unique_reactions = dict()

for row in messages_df['reactions']:
    try:
        for react in row:
            if react['reaction'] not in unique_reactions.keys():
                unique_reactions[react['reaction']] = dict()
                unique_reactions[react['reaction']]['cnt_messages'] = 0
                unique_reactions[react['reaction']]['cnt'] = 0
            unique_reactions[react['reaction']]['cnt_messages'] += 1
            unique_reactions[react['reaction']]['cnt'] += react['count']
    except:
        pass

In [8]:
reactions_df = pd.DataFrame(unique_reactions).T.reset_index().rename({'index': 'reaction'}, axis = 1)
display(reactions_df.shape)
reactions_df.head()

(73, 3)

,reaction,cnt_messages,cnt
0,👍,41491,3252292
1,👎,11117,185876
2,🤯,4957,73860
3,❤,19614,292295
4,🤔,9982,202470


In [9]:
reactions_df = reactions_df.query('cnt_messages > 500 or cnt > 5000').reset_index(drop=True)
display(reactions_df.shape)
reactions_df.head()

(31, 3)

,reaction,cnt_messages,cnt
0,👍,41491,3252292
1,👎,11117,185876
2,🤯,4957,73860
3,❤,19614,292295
4,🤔,9982,202470


In [10]:
reactions_df.set_index('reaction')['cnt'].to_dict()

{'👍': 3252292,
 '👎': 185876,
 '🤯': 73860,
 '❤': 292295,
 '🤔': 202470,
 '😢': 91386,
 '🔥': 451864,
 '👏': 36953,
 '🐳': 69281,
 '🍾': 5812,
 '❤\u200d🔥': 5641,
 '👌': 12239,
 '🤩': 6909,
 '😁': 434301,
 '🎉': 12603,
 '🥰': 2648,
 '🥴': 18227,
 '🤬': 83075,
 '😱': 83661,
 '🌚': 14845,
 '🌭': 27312,
 '🕊': 17637,
 '☃': 2636,
 '🍌': 11898,
 '💩': 7597,
 '🤣': 30176,
 '🤡': 9361,
 '😈': 5819,
 '💯': 4780,
 '\U0001fae1': 2300,
 '👀': 7414}

In [11]:
reactions_names_mapper = \
{'👍': 'thumb_up',
 '👎': 'thumb_down',
 '🤯': 'mindblowing',
 '❤': 'heart',
 '🤔': 'thinking',
 '😢': 'crying',
 '🔥': 'fire',
 '👏': 'clap',
 '🐳': 'whale',
 '🍾': 'champaign',
 '❤\u200d🔥': 'heart_fire',
 '👌': 'okay',
 '🤩': 'smile_stars',
 '😁': 'smiling',
 '🎉': 'cheers',
 '🥰': 'lovely_smile',
 '🥴': 'drunk_smile',
 '🤬': 'angry_smile',
 '😱': 'shicked_smile',
 '🌚': 'moos_smile',
 '🌭': 'hotdog',
 '🕊': 'free_bird',
 '☃': 'snowman',
 '🍌': 'banana',
 '💩': 'poop',
 '🤣': 'rofl_smile',
 '🤡': 'clown_smile',
 '😈': 'devil_smile',
 '💯': 'hundred',
 '\U0001fae1': 'head_arm_smile',
 '👀': 'eyes'}

In [12]:
messages_df[list(map(lambda x: x + '_react_cnt', reactions_names_mapper.values()))] = 0

In [13]:
unique_reactions = dict()

for idx in tqdm(messages_df.index):
    row = messages_df['reactions'][idx]
    try:
        for react in row:
            react_name = reactions_names_mapper[react['reaction']] + '_react_cnt'
            messages_df[react_name][idx] = react['count']
    except:
        pass

  0%|          | 0/90339 [00:00<?, ?it/s]

In [14]:
messages_df.head()

,message_id,channel_id,tg_message_id,date,text,views,forwards,reactions,thumb_up_react_cnt,thumb_down_react_cnt,...,free_bird_react_cnt,snowman_react_cnt,banana_react_cnt,poop_react_cnt,rofl_smile_react_cnt,clown_smile_react_cnt,devil_smile_react_cnt,hundred_react_cnt,head_arm_smile_react_cnt,eyes_react_cnt
0,552,1,4332,2023-11-02 12:31:28,"Центробанку, скорее всего, придётся поднять ст...",110394,401,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",317,147,...,0,0,0,0,0,0,0,0,0,0
1,551,1,4333,2023-11-02 14:33:01,Сбербанк продолжает рост: результаты по междун...,110495,76,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",322,21,...,0,0,0,0,0,0,0,0,0,0
2,550,1,4337,2023-11-02 15:56:40,Плохие новости сегодня вечером. СПБ Биржа попа...,125490,469,"[{'_': 'ReactionCount', 'reaction': '😢', 'coun...",215,20,...,0,0,0,0,0,0,0,0,0,0
3,549,1,4338,2023-11-03 07:59:27,📉 Вчера на российском рынке акций прошли распр...,323274,41,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",221,10,...,0,0,0,0,0,0,0,0,0,0
4,548,1,4339,2023-11-03 10:44:50,У Норникеля сразу две приятные новости: компан...,315471,240,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",350,21,...,0,0,0,0,0,0,0,0,0,0


### Промечаем сообщения

In [17]:
path = '../data/companies/companies.json'

with open(path, 'r') as f:
    companies_dict = json.load(f)

In [18]:
companies_list = list(companies_dict.keys())

messages_df[companies_list] = 0

In [19]:
for idx in tqdm(messages_df.index):
    text = messages_df['text'][idx]
    try:
        messages_df['text'][idx] = text.lower()
    except:
        pass

  0%|          | 0/90339 [00:00<?, ?it/s]

In [20]:
for idx in tqdm(messages_df.index):
    text = messages_df['text'][idx]
    # try:
    for company in companies_list:
        for name in companies_dict[company]['names']:
            pos = text.find(name)
            if pos != -1:
                messages_df[company][idx] = 1
    # except:
    #     pass

  0%|          | 0/90339 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out


In [21]:
messages_df.head()

,message_id,channel_id,tg_message_id,date,text,views,forwards,reactions,thumb_up_react_cnt,thumb_down_react_cnt,...,rusneft,sfi,nkhp,mkb,ovk,rosagro,yandex,qiwi,polymetal,moex
0,552,1,4332,2023-11-02 12:31:28,"центробанку, скорее всего, придётся поднять ст...",110394,401,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",317,147,...,0,0,0,0,0,0,0,0,0,0
1,551,1,4333,2023-11-02 14:33:01,сбербанк продолжает рост: результаты по междун...,110495,76,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",322,21,...,0,0,0,0,0,0,0,0,0,1
2,550,1,4337,2023-11-02 15:56:40,плохие новости сегодня вечером. спб биржа попа...,125490,469,"[{'_': 'ReactionCount', 'reaction': '😢', 'coun...",215,20,...,0,0,0,0,0,0,0,0,0,0
3,549,1,4338,2023-11-03 07:59:27,📉 вчера на российском рынке акций прошли распр...,323274,41,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",221,10,...,0,0,0,0,0,0,0,0,0,1
4,548,1,4339,2023-11-03 10:44:50,у норникеля сразу две приятные новости: компан...,315471,240,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",350,21,...,0,0,0,0,0,0,0,0,0,0


In [22]:
columns = ['message_id', 'channel_id', 'tg_message_id', 'date', 'text', 'views',
       'forwards', 'reactions', 'thumb_up_react_cnt', 'thumb_down_react_cnt',
       'mindblowing_react_cnt', 'heart_react_cnt', 'thinking_react_cnt',
       'crying_react_cnt', 'fire_react_cnt', 'clap_react_cnt',
       'whale_react_cnt', 'champaign_react_cnt', 'heart_fire_react_cnt',
       'okay_react_cnt', 'smile_stars_react_cnt', 'smiling_react_cnt',
       'cheers_react_cnt', 'lovely_smile_react_cnt', 'drunk_smile_react_cnt',
       'angry_smile_react_cnt', 'shicked_smile_react_cnt',
       'moos_smile_react_cnt', 'hotdog_react_cnt', 'free_bird_react_cnt',
       'snowman_react_cnt', 'banana_react_cnt', 'poop_react_cnt',
       'rofl_smile_react_cnt', 'clown_smile_react_cnt',
       'devil_smile_react_cnt', 'hundred_react_cnt',
       'head_arm_smile_react_cnt', 'eyes_react_cnt']

pairs_df = pd.DataFrame(columns = columns)
pairs_df['company'] = None
pairs_df['ticker'] = None

In [23]:
for company in companies_list:
    cur_df = messages_df[messages_df[company] == 1][columns]
    cur_df['company'] = company
    cur_df['ticker'] = companies_dict[company]['ticker']
    pairs_df = pd.concat([pairs_df, cur_df])

In [24]:
pairs_df = pairs_df.reset_index(drop = True).drop('reactions', axis = 1)

In [25]:
pairs_df.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,banana_react_cnt,poop_react_cnt,rofl_smile_react_cnt,clown_smile_react_cnt,devil_smile_react_cnt,hundred_react_cnt,head_arm_smile_react_cnt,eyes_react_cnt,company,ticker
0,1675,3,22642,2023-10-06 16:37:06,все больше компаний рф заявляют о планах по ip...,65009,84,301,14,0,...,0,0,0,0,0,0,0,0,crystal,KLVZ
1,1474,3,22861,2023-10-31 11:41:23,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0,0,0,0,0,0,0,0,crystal,KLVZ
2,758,3,23707,2024-02-20 18:20:41,📺 алкогольная группа «кристалл» уже в этот чет...,69592,119,146,86,0,...,0,0,0,0,0,0,0,0,crystal,KLVZ


In [26]:
path = '../data/messages/messages_info.csv'
pairs_df.to_csv(path)